In [60]:

from __future__ import division, print_function
import sympy as sm
from sympy import trigsimp
from sympy import latex

import matplotlib.pyplot as plt
import sympy.physics.mechanics as me
import mpmath as mp
import symbtools as st
import numpy as np
import symbtools.modeltools as mt
import symbtools.noncommutativetools as nct
from scipy.integrate import odeint
#import pytrajectory as pytr
#import pycartan as pc
from sympy.solvers import solve

#from pytrajectory import log
#log.console_handler.setLevel(10)

from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)

import dill
#import ipydex

#definingymbolic variables
#q = st.symb_vector('q:2')
#qdot = st.time_deriv(q, q)
#qdd = st.time_deriv(qdot, qdot)
q0,q1=me.dynamicsymbols('q0 q1')
qdot0, qdot1=me.dynamicsymbols('q0 q1',1)
q=[q0, q1]
qdot=[qdot0, qdot1]
#st.make_global(q0, qdot0)

f = sm.symbols('f')
m = sm.symbols('m:2')
J = sm.symbols('J:2')
l = sm.symbols('l')
a = sm.symbols('a')
d = sm.symbols('d')
g, t = sm.symbols('g t')

# defining reference frames and points
In_frame = me.ReferenceFrame('In_frame')  #reference frame with velocity 0
O = me.Point('O')
O.set_vel(In_frame, 0)

# Mass center of the cart
C0 = me.Point('C0')
C0.set_pos(O, q[0] * In_frame.x)
C0.set_vel(In_frame, qdot[0] * In_frame.x)

#cart frame
cart_frame = In_frame.orientnew('L0', 'Axis', [q[0], In_frame.x])
cart_frame.set_ang_vel(In_frame, 0)

#endulum frame
pen_frame = In_frame.orientnew('L1', 'Axis', [ sm.pi/2-q[1], In_frame.z])
pen_frame.set_ang_vel(In_frame, -qdot[1] * In_frame.z)
#mass center of pendulum
s = C0.locatenew('a', a * pen_frame.x)
s.v2pt_theory(C0, In_frame, pen_frame)

# finding velocities in inertial reference frame --> use them to find kinetic energy
v0 = C0.vel(In_frame)
v1 = s.vel(In_frame)
T = 1 / 2 * m[0] * v0.dot(v0) + 1 / 2 * m[1] * v1.dot(v1) + 1 / 2 * J[1] * qdot[1]**2

#v00=qdot[0]
#v11_sq=(qdot[0]+a*qdot[1]*sm.cos(q[1]))**2 + (a*qdot[1]*sm.sin(q[1]))**2

T1=1/2 * m[0] * v00**2 +1/2 * m[1] * v11_sq +1/2 *J[1]*qdot[1]**2
# potential energy
h = s.pos_from(O).dot(In_frame.y)
#V = m[0] * g * h - 1/2 * d * qdot[1]**2
#V = m[0] * g * h
V=sm.S(0)
tau = d * qdot[1] + m[0]*a*g*sm.sin(q1)

params = [m[0], m[1], J[0], J[1], a, d, g]
st.make_global(params)


In [61]:
T

0.5*J1*q1'**2 + 0.5*m0*q0'**2 + 0.5*m1*(a**2*q1'**2 + 2*a*cos(q1)*q0'*q1' + q0'**2)

In [72]:
tau

a*g*m0*sin(q_t1) + d*q_tdot1

In [73]:
q_t = st.symb_vector('q_t:2')
qdot_t = st.time_deriv(q_t, q_t)
qdd_t = st.time_deriv(qdot_t, qdot_t)
st.make_global(q_t, qdot_t)

In [74]:
qlist=[(q0, q_t[0]), (q1, q_t[1]), (qdot0, qdot_t[0]),(qdot1, qdot_t[1])]
qdict=dict(qlist)

T=T.subs(qdict)
#V=V.subs(qdict)
tau=tau.subs(qdict)

In [65]:

mod = mt.generate_symbolic_model(T, V, q_t, [f, tau])
mod.eqns.simplify()

mod.calc_state_eq(simplify=True)
xx = mod.xx
fx = mod.f
gx = mod.g


qq=sm.Matrix([q_t, qdot_t])

#fx_lin=fx.jacobian(qq)
#gx_lin=G.jacobian(qq)

equilib_point = sm.Matrix([1.0,sm.pi/6 , 1, 1])

parameter_values = [(g, 9.81), (a, 0.2), (d, 0.0), (m[0], 3.34),
                    (m[1], 0.8512), (J[0], 0), (J[1], 0.01980)]

parameters = list(map(lambda a,b :(a,b),qq, equilib_point)) + parameter_values
param_dict=dict(parameters)


#fx_lin.subs(param_dict)
#gx_lin.subs(param_dict)

In [71]:
fx.subs(param_dict).evalf()

Matrix([
[              1.0],
[              1.0],
[-2.34606686607372],
[ 67.2713140077747]])

In [38]:
fx_00=fx[0].diff(q_t[0])
fx_01=fx[0].diff(q_t[1])
fx_02=fx[0].diff(qdot_t[0])
fx_03=fx[0].diff(qdot_t[1])

fx_10=fx[1].diff(q_t[0])
fx_11=fx[1].diff(q_t[1])
fx_12=fx[1].diff(qdot_t[0])
fx_13=fx[1].diff(qdot_t[1])

fx_20=fx[2].diff(q_t[0])
fx_21=fx[2].diff(q_t[1])
fx_22=fx[2].diff(qdot_t[0])
fx_23=fx[2].diff(qdot_t[1])

fx_30=fx[3].diff(q_t[0])
fx_31=fx[3].diff(q_t[1])
fx_32=fx[3].diff(qdot_t[0])
fx_33=fx[3].diff(qdot_t[1])

In [39]:
fx_lin=sm.Matrix([[fx_00, fx_01, fx_02, fx_03], 
                 [fx_10, fx_11, fx_12, fx_13],
                 [fx_20, fx_21, fx_22, fx_23],
                 [fx_30, fx_31, fx_32, fx_33]])

In [40]:
fx_lin.subs(param_dict)

Matrix([
[0,                0, 1, 0],
[0,                0, 0, 1],
[0, 5.67138717420427, 0, 0],
[0, 139.625927658159, 0, 0]])

In [41]:
gx

Matrix([
[                                                                               0],
[                                                                               0],
[-(1.0*J1 + 1.0*a**2*m1)/(a**2*m1**2*cos(q_t1)**2 - (J1 + a**2*m1)*(1.0*m0 + m1))],
[    -1.0*a*m1*cos(q_t1)/(a**2*m1**2*cos(q_t1)**2 - (J1 + a**2*m1)*(1.0*m0 + m1))]])

In [42]:
gx_20=gx[2].diff(q_t[0])
gx_21=gx[2].diff(q_t[1])
gx_22=gx[2].diff(qdot_t[0])
gx_23=gx[2].diff(qdot_t[1])

gx_30=gx[3].diff(q_t[0])
gx_31=gx[3].diff(q_t[1])
gx_32=gx[3].diff(qdot_t[0])
gx_33=gx[3].diff(qdot_t[1])

gx_lin=sm.Matrix([[0., 0., 0., 0.], 
                 [0., 0., 0., 0.],
                 [gx_20, gx_21, gx_22, gx_23],
                 [gx_30, gx_31, gx_32, gx_33]])

In [23]:
gx_lin.simplify()
gx_lin

Matrix([
[0.0,                                                                                                                                      0.0, 0.0, 0.0],
[0.0,                                                                                                                                      0.0, 0.0, 0.0],
[  0,                   -4.0*a**2*m1**2*(J1 + a**2*m1)*sin(2*q_t1)/(2*J1*m0 + 2*J1*m1 + 2*a**2*m0*m1 - a**2*m1**2*cos(2*q_t1) + a**2*m1**2)**2,   0,   0],
[  0, -1.0*a*m1*(a**2*m1**2*cos(q_t1)**2 + (J1 + a**2*m1)*(1.0*m0 + m1))*sin(q_t1)/(a**2*m1**2*cos(q_t1)**2 - (J1 + a**2*m1)*(1.0*m0 + m1))**2,   0,   0]])

In [ ]:
gx_lin.simplify()
gx_lin

In [24]:
gx_lin.subs(param_dict)

Matrix([
[0.0, 0.0, 0.0, 0.0],
[0.0, 0.0, 0.0, 0.0],
[  0,   0,   0,   0],
[  0,   0,   0,   0]])

In [27]:

'''
q[0] = sm.Function('q[0]')
q[1] = sm.Function('q[1]')
'''
L=T-V
dL_dqdot0=sm.diff(L, qdot[0])
dL_dqdot1=sm.diff(L, qdot[1])
dL_dq0=sm.diff(L, q[0])
dL_dq1=sm.diff(L, q[1])

dt_dL_dqdot0=st.time_deriv(dL_dqdot0, [q[1]]).simplify()
dt_dL_dqdot1=sm.diff(dL_dqdot1, t).simplify()

exp1=dt_dL_dqdot0-dL_dq0-f
exp2=dt_dL_dqdot1-dL_dq1


In [28]:
xx

Matrix([
[   q_t0],
[   q_t1],
[q_tdot0],
[q_tdot1]])